### Jupyter Notebook, contains same code as `train.py`

In [1]:
import gym
import numpy as np
from collections import deque
from gym import spaces
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from custom_wrappers import *
from utils import *
from build_model import build_dq_model
from memory import Memory

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


2022-05-11 23:00:03.554703: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-11 23:00:03.554724: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/yash/Desktop/Courses/CS2470/Final_Project/repo/proj_env_2/lib/python3.7/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/home/yash/Desktop/Courses/CS2470/Final_Project/repo/proj_env_2/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:23: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
/home/yash/Desktop/Courses/CS2470/Final_Project/repo/proj

Current Directory -> /home/yash
Num GPUs Available:  0


2022-05-11 23:00:06.689746: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-11 23:00:06.689771: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-11 23:00:06.689784: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (yash-ThinkPad-L390): /proc/driver/nvidia/version does not exist


In [2]:
def choose_action(q_net, state, epsilon, num_actions):
    if epsilon > np.random.rand(1)[0]:
        action = np.random.choice(num_actions)
    else:
        state_t = tf.expand_dims(tf.convert_to_tensor(state), 0)
        action = tf.argmax(q_net(state_t, training=False)[0]).numpy()
    return action

In [3]:
# Using the wrappers for the environment
env = gym.make("ALE/Riverraid-v5")
env = ObservationWrapper(RewardWrapper(ActionWrapper(ConcatObs(FireResetEnv(env), k=4, DEATH_REWARD=1000)), CLIP=False, SCALE=True), GRAYSCALE=True, NORMALIZE=True)
obs = env.reset()

A.L.E: Arcade Learning Environment (version 0.7.5+db37282)
[Powered by Stella]
2022-05-11 23:00:09.622973: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Build the model
hidden_size = 640
num_actions = 18
q_net = build_dq_model(input_shape=obs.shape, hidden_size=hidden_size, num_actions=num_actions)
q_net_target = build_dq_model(input_shape=obs.shape, hidden_size=hidden_size, num_actions=num_actions)
loss_func = keras.losses.Huber()
optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)
experience = Memory(length=10000)

In [5]:
# Defining hyperparameters
discount_factor = 0.99
batch_size = 32
n_steps_episode = 10000
rolling_reward = 0
episode_ix = 0
observation_ix = 0
random_till = 3000
eps_decay = 0.9995
epsilon = 1.0
eps_threshold = 0.1
weight_update_freq = 10
target_update_freq = 500
n_episodes = 50000

In [6]:
# Kick off training
write_to_log("Starting Training for new implementation", include_blank_line=True)
reward_history = np.zeros(n_episodes)
num_steps_history = np.zeros(n_episodes)
SWITCHED_TO_GREEDY = False

In [ ]:
while episode_ix < n_episodes:
    state = np.array(env.reset())
    ep_total_reward = 0

    if episode_ix in [500, 2000, 5000, 10000, 20000, 50000]:
        # print("Episode: ", episode_ix)
        write_to_log("Saving model at " + str(episode_ix))
        curr_model_name = "curr_model_rev_" + str(episode_ix)
        curr_model_name_target = "curr_model_rev_target_" + str(episode_ix)
        save_models_and_arrays(curr_model_name, curr_model_name_target, 
                            q_net, q_net_target,
                            reward_history[:episode_ix], 
                            num_steps_history[:episode_ix])

    for _ in range(1, n_steps_episode):
        observation_ix += 1

        if observation_ix < random_till:
            # Still exploring, not following policy
            action = np.random.choice(num_actions)
        else:
            # Take action as suggested by policy
            if not SWITCHED_TO_GREEDY:
                write_to_log("Switching to Greedy, at observation ", observation_ix)
                SWITCHED_TO_GREEDY = True
            action = choose_action(q_net, state, epsilon, num_actions)

        # Take action and get new observations
        state_next, reward, done, _ = env.step(action)
        state_next = np.array(state_next)

        ep_total_reward += reward

        # Update Experience with experience tuple and then update state
        experience.append((state, state_next, action, reward, done))
        state = state_next

        # Decay epsilon till threshold
        epsilon = max(epsilon*eps_decay, eps_threshold)

        if len(experience) > batch_size and observation_ix % weight_update_freq == 0:

            state_sample, state_next_sample, rewards_sample, action_sample, done_sample = \
                experience.sample_memory(batch_size)

            # Calculate Q values as sum of reward and discounted future rewards and handle final state
            future_rewards = q_net_target.predict(state_next_sample)
            updated_q_vals = rewards_sample + (discount_factor * tf.reduce_max(future_rewards, axis=1))
            updated_q_vals = updated_q_vals * (1 - done_sample) - done_sample

            masked_action = tf.one_hot(tf.cast(action_sample, dtype=tf.int32), num_actions)

            with tf.GradientTape() as tape:
                q_vals = q_net(state_sample)
                q_action = tf.reduce_sum(tf.multiply(q_vals, masked_action), axis=1)
                loss = loss_func(updated_q_vals, q_action)

            # Backpropagation
            grads = tape.gradient(loss, q_net.trainable_variables)
            optimizer.apply_gradients(zip(grads, q_net.trainable_variables))

        if observation_ix % target_update_freq == 0:
            # Update Target QNet
            q_net_target.set_weights(q_net.get_weights())
            template = "running reward: {:.2f} at episode {}, frame count {}"
            print(template.format(rolling_reward, episode_ix, observation_ix))

        if done:
            break

    reward_history[episode_ix] = ep_total_reward
    rolling_reward = np.mean(reward_history[:episode_ix])

    episode_ix += 1


In [ ]:
# Save final model
curr_model_name = "final_curr_rev_model_" + str(episode_ix)
curr_model_name_target = "final_curr_model_rev_target_" + str(episode_ix)
save_models_and_arrays(curr_model_name, curr_model_name_target, 
                    q_net, q_net_target,
                    reward_history, num_steps_history)
write_to_log("Training for new implementation complete")